In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import re
import string
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer

/tmp/ipykernel_11335/1685585545.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
nltk.download('snowball_data')
stemmer = SnowballStemmer('spanish')

[nltk_data] Downloading package snowball_data to
[nltk_data]     /home/azul/nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!


In [3]:
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /home/azul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv('sentiment_integrado.csv')

In [5]:
df["sentimiento"] = df["sentimiento"].apply(lambda x: x if x != "Indefinido" else "Neutral")

In [6]:
mappeo_sent = { "Neutral": 0, "Negativo": 1, "Positivo": 2}
df["sentimiento"] = df["sentimiento"].apply(lambda x: mappeo_sent[x])

In [7]:
train, test_valid_data = train_test_split(df[['texto', 'sentimiento']].dropna(), test_size=0.4, random_state=42)
test, validation = train_test_split(test_valid_data, test_size=0.5, random_state=42)

In [8]:
def procesar_tweets(tweet):
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stop_words and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return ' '.join(tweets_clean)

In [9]:
train.head(10)

,texto,sentimiento
10221,"@Manuellflorod Bienvenida (triste) realidad, a...",1
9565,Estar en los brazos de mi novio es lo único qu...,2
7555,"Todos tenemos un pasado @ahorapodemos ,que se ...",0
7713,@ahorapodemos @ierrejon Tambien que las empres...,2
2825,En Islandia: juicio a los responsables de la ...,0
1038,El tono duro y sin concesiones de la réplica d...,1
6331,Comienza su intervención en el acto de cierre ...,0
10,Bdías. EM no se ira de puente. Si vosotros os ...,2
2647,"#FF @ArturoGomezQuij, por demostrar en su penú...",2
5868,"""La herencia del @PSOE : trabajos a ratos y 5 ...",1


In [10]:
train.head(10).texto.apply(lambda x: procesar_tweets(x))

10221                      bienven trist realid andam mism
9565                          braz novi unic quier necesit
7555                                               pas dig
7713     tambi empres independient contact si estub int...
2825     islandi juici respons crisis negat pag deud el...
1038     ton dur concesion replic rajoy amaiur anunci c...
6331     comienz intervencion acto cierr campañ siguel ...
10       bdi em ira puent si vais dejeis llev tablet pc...
2647     ff demostr penultim tweet prejuici oportun act...
5868     herenci trabaj rat 5 eur hor editorial 20.30 h...
Name: texto, dtype: object

In [12]:
train['texto_procesado'] = train.texto.apply(lambda x: procesar_tweets(x))
validation['texto_procesado'] = validation.texto.apply(lambda x: procesar_tweets(x))
test['texto_procesado'] = test.texto.apply(lambda x: procesar_tweets(x))

In [18]:
train

,texto,sentimiento,texto_procesado
10221,"@Manuellflorod Bienvenida (triste) realidad, a...",1,"[bienven, trist, realid, andam, mism]"
9565,Estar en los brazos de mi novio es lo único qu...,2,"[braz, novi, unic, quier, necesit]"
7555,"Todos tenemos un pasado @ahorapodemos ,que se ...",0,"[pas, dig]"
7713,@ahorapodemos @ierrejon Tambien que las empres...,2,"[tambi, empres, independient, contact, si, est..."
2825,En Islandia: juicio a los responsables de la ...,0,"[islandi, juici, respons, crisis, negat, pag, ..."
...,...,...,...
11285,"No, solo quiero bailarte la medusa loca en pri...",2,"[sol, quier, bailart, medus, loc, priv]"
11965,@cuervotinelli lo único que te puse fue que qu...,1,"[unic, pus, quer, jug, lol, tortug, retwitt, m..."
5390,La Audiencia Nacional condena a 20 años de pri...,1,"[audienci, nacional, conden, 20, años, prision..."
860,RT @eP_Titulares: #Sociedad Muere apuñalada la...,1,"[socied, muer, apuñal, propietari, zapat, banyol]"


In [13]:
train.to_csv('processed_tweets_stemm/train.csv')
validation.to_csv('processed_tweets_stemm/validation.csv')
test.to_csv('processed_tweets_stemm/test.csv')